In [1]:
import requests
from bs4 import BeautifulSoup
import json
import time # Dùng để sleep chương trình
import pandas as pd # Dùng để đọc và hiển thị file csv (Pandas sẽ được học chi tiết ở buổi tới)
import datetime as dt # Dùng để xử lý dữ liệu thời gian
import re

In [56]:
Total_pages=10
req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}

with requests.Session() as s:
    page =1
    url = 'https://www.zillow.com/manchester-nh/sold/?searchQueryState=%7B"pagination"%3A%7B%7D%2C"mapBounds"%3A%7B"west"%3A-71.53618812561035%2C"east"%3A-71.34221076965332%2C"south"%3A42.96100798811988%2C"north"%3A43.06192743848626%7D%2C"regionSelection"%3A%5B%7B"regionId"%3A5772%2C"regionType"%3A6%7D%5D%2C"isMapVisible"%3Atrue%2C"filterState"%3A%7B"sort"%3A%7B"value"%3A"globalrelevanceex"%7D%2C"fsba"%3A%7B"value"%3Afalse%7D%2C"fsbo"%3A%7B"value"%3Afalse%7D%2C"nc"%3A%7B"value"%3Afalse%7D%2C"fore"%3A%7B"value"%3Afalse%7D%2C"cmsn"%3A%7B"value"%3Afalse%7D%2C"auc"%3A%7B"value"%3Afalse%7D%2C"pmf"%3A%7B"value"%3Afalse%7D%2C"pf"%3A%7B"value"%3Afalse%7D%2C"rs"%3A%7B"value"%3Atrue%7D%2C"ah"%3A%7B"value"%3Atrue%7D%7D%2C"isListVisible"%3Atrue%2C"mapZoom"%3A13%7D'
    while(page < Total_pages):
        time.sleep(1)
        r = s.get(url, headers=req_headers)
        soup = BeautifulSoup(r.content, 'html.parser')
        detailLink = soup.find_all('a',{'class':'list-card-link list-card-link-top-margin'})
        prices = soup.find_all('div',{'class':'list-card-price'})
        page+=1
        url ='https://www.zillow.com/manchester-nh/sold/' +str(page)+'_p/'
        for link in detailLink:
            print(link)
            address = link.text
            detail = s.get(link['href'],headers=req_headers)
            soup1 = BeautifulSoup(detail.content, 'html.parser')
            bbl = soup1.find_all('span',{'class':'ds-bed-bath-living-area'},limit =3) # bed-bad-livingarea
            bed_bad_livingarea =''
            for i in bbl:
                bed_bad_livingarea+= i.text +'\t'
            print(bed_bad_livingarea)
            heating = soup1.find("span",string=re.compile("Heating:"))
            if heating is not None:
                heating = heating.find_next_sibling('span').text
                print(heating)
            basement = soup1.find("span",string=re.compile("Basement"))
            if basement is not None:
                basement = basement.find_next('span').text.split(': ')[1]
                print(basement)
            yearbuild = soup1.find("span",string=re.compile("Condition"))
            if yearbuild is not None:
                yearbuild = yearbuild.find_next('span').text.split(': ')[1]
                print(yearbuild)
            fireplace = soup1.find("h5",string=re.compile("Property details"))
            if fireplace is not None:
                fireplace = fireplace.find_previous('span').text.split(': ')[1]
                print(fireplace)
            garage = soup1.find("span",string=re.compile("Property"))
            if garage is not None:
                garage = garage.find_previous('span').text.split(': ')[1]
                print(garage)
            
            #fw.write(address +'\t'+ bed_bad_livingarea +'\t'+ provider +'\t'+ rating +'\t'+ num_ratings + '\n')
        

<a class="list-card-link list-card-link-top-margin" href="https://www.zillow.com/homedetails/55-River-Rd-APT-7A-Manchester-NH-03104/95349869_zpid/" tabindex="0"><address class="list-card-addr">55 River Rd APT 7A, Manchester, NH 03104</address></a>
2 bd	2 ba	761 sqft	
Baseboard, Electric
None
1973
761 sqft
2
<a class="list-card-link list-card-link-top-margin" href="https://www.zillow.com/homedetails/10-Estate-Dr-Manchester-NH-03104/74321535_zpid/" tabindex="0"><address class="list-card-addr">10 Estate Dr, Manchester, NH 03104</address></a>
3 bd	2 ba	1,597 sqft	
Electric, Oil
Yes
1978
Yes
1
<a class="list-card-link list-card-link-top-margin" href="https://www.zillow.com/homedetails/415-Calef-Rd-Manchester-NH-03103/74313545_zpid/" tabindex="0"><address class="list-card-addr">415 Calef Rd, Manchester, NH 03103</address></a>
3 bd	2 ba	1,092 sqft	
Baseboard, Electric, Gas, Oil
Yes
1955
1,092 sqft
3
<a class="list-card-link list-card-link-top-margin" href="https://www.zillow.com/homedetails/4

IndexError: list index out of range